# Postprocess wrfout for offshore wind
Last Updated by Jeffrey Sward on October 27, 2020

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

# built in python modules
import datetime
import os
import inspect
import sys

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import wrf

# Import the pvlib module
if sys.platform == 'linux':
    sys.path.append('/home/jsward/Documents/01_Research/01_Renewable_Analysis/WRF/pvlib-python')
import pvlib
from pvlib.wrfcast import WRF
from optwrf import postwrf

In [2]:
# {ath to WRF data file
datapath = os.path.join('/Users/swardy9230/Box Sync/01_Research/01_Renewable_Analysis/Wind Resource Analysis/wrfout/19mp4lw4sw7lsm8pbl99cu', 'wrfout_d01_2011-07-17')

# Read in the wrfout file using the netCDF4.Dataset method (the xarray method doesn't allow us to use wrf.getvar
netcdf_data = netCDF4.Dataset(datapath)
# netcdf_data = xr.open_dataset(datapath)
# netcdf_data

In [3]:
# Get global attributes from the NetCDF Dataset
wrfattrs_names = netcdf_data.ncattrs()
wrfattrs = wrf.extract_global_attrs(netcdf_data, wrfattrs_names)

# Create an xarray.Dataset from the wrf qurery_variables.
query_variables = [
    'times',
    'T2',
    'CLDFRA',
    'COSZEN',
    'SWDDNI',
    'SWDDIF',
    'height_agl',
#     'wspd',
#     'wdir',
        ]
first = True
for key in query_variables:
    var = wrf.getvar(netcdf_data, key, timeidx=wrf.ALL_TIMES)
    if first:
        met_data = var
#         print(f'{met_data}\n')
        first = False
    else:
        with xr.set_options(keep_attrs=True):
            try:
                met_data = xr.merge([met_data, var])
            except ValueError: 
                print(key)
                var = xr.DataArray.reset_coords(var, ['key'], drop=True)
#                 met_data = met_data.drop_vars('Time')
                met_data = xr.merge([met_data, var])
        
variables = {
    'times': 'Times',
    'XLAT': 'lat',
    'XLONG': 'lon',
    'T2': 'temp_air',
    'CLDFRA': 'cloud_fraction',
    'COSZEN': 'cos_zenith',
    'SWDDNI': 'dni',
    'SWDDIF': 'dhi',
    'height_agl': 'height_agl',
#     'wspd': 'wspd',
#     'wdir': 'wdir',
}
met_data = xr.Dataset.rename(met_data, variables)
met_data = xr.Dataset.reset_coords(met_data, ['XTIME'], drop=True)
times = met_data.Times
# met_data = xr.Dataset.set_coords(met_data, ['Times'])
# met_data = xr.Dataset.reset_coords(met_data, ['Times'], drop=True)
ntimes = met_data.sizes['Time']
nlat = met_data.sizes['south_north']
nlon = met_data.sizes['west_east']
met_data = met_data.assign_attrs(wrfattrs)
met_data

<xarray.Dataset>
Dimensions:         (Time: 40, bottom_top: 35, south_north: 191, west_east: 191)
Coordinates:
  * Time            (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    lon             (south_north, west_east) float32 -95.43335 ... -64.11832
    lat             (south_north, west_east) float32 30.952759 ... 47.507103
Dimensions without coordinates: bottom_top, south_north, west_east
Data variables:
    Times           (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    temp_air        (Time, south_north, west_east) float32 308.1148 ... 292.3...
    cloud_fraction  (Time, bottom_top, south_north, west_east) float32 0.0 .....
    cos_zenith      (Time, south_north, west_east) float32 0.0 ... 0.84530807
    dni             (Time, south_north, west_east) float32 0.0 0.0 ... 943.1471
    dhi             (Time, south_north, west_east) float32 0.0 0.0 ... 53.327393
    height_agl      (Time, bottom_top, south_north, west_east) float32 10.885...
Attributes:
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      2011-07-17_00:00:00
    SIMULATION_START_DATE:           2011-07-17_00:00:00
    WEST-EAST_GRID_DIMENSION:        192
    SOUTH-NORTH_GRID_DIMENSION:      192
    BOTTOM-TOP_GRID_DIMENSION:       36
    DX:                              12000.0
    DY:                              12000.0
    AERCU_OPT:                       0
    AERCU_FCT:                       1.0
    IDEAL_CASE:                      0
    DIFF_6TH_SLOPEOPT:               0
    AUTO_LEVELS_OPT:                 2
    DIFF_6TH_THRESH:                 0.1
    DZBOT:                           50.0
    DZSTRETCH_S:                     1.3
    DZSTRETCH_U:                     1.1
    SKEBS_ON:                        0
    SPEC_BDY_FINAL_MU:               1
    USE_Q_DIABATIC:                  0
    GRIDTYPE:                        C
    DIFF_OPT:                        1
    KM_OPT:                          4
    DAMP_OPT:                        0
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      19
    RA_LW_PHYSICS:                   4
    RA_SW_PHYSICS:                   4
    SF_SFCLAY_PHYSICS:               1
    SF_SURFACE_PHYSICS:              7
    BL_PBL_PHYSICS:                  8
    CU_PHYSICS:                      99
    SF_LAKE_PHYSICS:                 0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      0
    GRID_FDDA:                       0
    GFDDA_INTERVAL_M:                0
    GFDDA_END_H:                     0
    GRID_SFDDA:                      0
    SGFDDA_INTERVAL_M:               0
    SGFDDA_END_H:                    0
    HYPSOMETRIC_OPT:                 2
    USE_THETA_M:                     1
    GWD_OPT:                         0
    SF_URBAN_PHYSICS:                0
    SF_SURFACE_MOSAIC:               0
    SF_OCEAN_PHYSICS:                0
    SHCU_PHYSICS:                    0
    MFSHCONV:                        0
    FEEDBACK:                        1
    SMOOTH_OPTION:                   0
    SWRAD_SCAT:                      1.0
    W_DAMPING:                       0
    DT:                              30.0
    RADT:                            15.0
    BLDT:                            0.0
    CUDT:                            0.0
    AER_OPT:                         0
    SWINT_OPT:                       1
    AER_TYPE:                        1
    AER_AOD550_OPT:                  1
    AER_ANGEXP_OPT:                  1
    AER_SSA_OPT:                     1
    AER_ASY_OPT:                     1
    AER_AOD550_VAL:                  0.12
    AER_ANGEXP_VAL:                  1.3
    AER_SSA_VAL:                     0.85
    AER_ASY_VAL:                     0.9
    MOIST_ADV_OPT:                   1
    SCALAR_ADV_OPT:                  1
    TKE_ADV_OPT:                     1
    DIFF_6TH_OPT:                  

In [5]:
# Process the data using the WRF forecast model methods
fm = WRF()
# met_data = fm.process_data(met_data)
temp_air = fm.kelvin_to_celsius(met_data['temp_air'])
ghi = fm.dni_and_dhi_to_ghi(met_data['dni'], met_data['dhi'], met_data['cos_zenith'])
met_data

<xarray.Dataset>
Dimensions:         (Time: 40, bottom_top: 35, south_north: 191, west_east: 191)
Coordinates:
  * Time            (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    lon             (south_north, west_east) float32 -95.43335 ... -64.11832
    lat             (south_north, west_east) float32 30.952759 ... 47.507103
Dimensions without coordinates: bottom_top, south_north, west_east
Data variables:
    Times           (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    temp_air        (Time, south_north, west_east) float32 308.1148 ... 292.3...
    cloud_fraction  (Time, bottom_top, south_north, west_east) float32 0.0 .....
    cos_zenith      (Time, south_north, west_east) float32 0.0 ... 0.84530807
    dni             (Time, south_north, west_east) float32 0.0 0.0 ... 943.1471
    dhi             (Time, south_north, west_east) float32 0.0 0.0 ... 53.327393
    height_agl      (Time, bottom_top, south_north, west_east) float32 10.885...
Attributes:
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      2011-07-17_00:00:00
    SIMULATION_START_DATE:           2011-07-17_00:00:00
    WEST-EAST_GRID_DIMENSION:        192
    SOUTH-NORTH_GRID_DIMENSION:      192
    BOTTOM-TOP_GRID_DIMENSION:       36
    DX:                              12000.0
    DY:                              12000.0
    AERCU_OPT:                       0
    AERCU_FCT:                       1.0
    IDEAL_CASE:                      0
    DIFF_6TH_SLOPEOPT:               0
    AUTO_LEVELS_OPT:                 2
    DIFF_6TH_THRESH:                 0.1
    DZBOT:                           50.0
    DZSTRETCH_S:                     1.3
    DZSTRETCH_U:                     1.1
    SKEBS_ON:                        0
    SPEC_BDY_FINAL_MU:               1
    USE_Q_DIABATIC:                  0
    GRIDTYPE:                        C
    DIFF_OPT:                        1
    KM_OPT:                          4
    DAMP_OPT:                        0
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      19
    RA_LW_PHYSICS:                   4
    RA_SW_PHYSICS:                   4
    SF_SFCLAY_PHYSICS:               1
    SF_SURFACE_PHYSICS:              7
    BL_PBL_PHYSICS:                  8
    CU_PHYSICS:                      99
    SF_LAKE_PHYSICS:                 0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      0
    GRID_FDDA:                       0
    GFDDA_INTERVAL_M:                0
    GFDDA_END_H:                     0
    GRID_SFDDA:                      0
    SGFDDA_INTERVAL_M:               0
    SGFDDA_END_H:                    0
    HYPSOMETRIC_OPT:                 2
    USE_THETA_M:                     1
    GWD_OPT:                         0
    SF_URBAN_PHYSICS:                0
    SF_SURFACE_MOSAIC:               0
    SF_OCEAN_PHYSICS:                0
    SHCU_PHYSICS:                    0
    MFSHCONV:                        0
    FEEDBACK:                        1
    SMOOTH_OPTION:                   0
    SWRAD_SCAT:                      1.0
    W_DAMPING:                       0
    DT:                              30.0
    RADT:                            15.0
    BLDT:                            0.0
    CUDT:                            0.0
    AER_OPT:                         0
    SWINT_OPT:                       1
    AER_TYPE:                        1
    AER_AOD550_OPT:                  1
    AER_ANGEXP_OPT:                  1
    AER_SSA_OPT:                     1
    AER_ASY_OPT:                     1
    AER_AOD550_VAL:                  0.12
    AER_ANGEXP_VAL:                  1.3
    AER_SSA_VAL:                     0.85
    AER_ASY_VAL:                     0.9
    MOIST_ADV_OPT:                   1
    SCALAR_ADV_OPT:                  1
    TKE_ADV_OPT:                     1
    DIFF_6TH_OPT:                  

In [6]:
# Process the data using the wrf-python package
height = wrf.getvar(netcdf_data, "height_agl", wrf.ALL_TIMES, units='m')
wspd = wrf.getvar(netcdf_data, 'wspd_wdir', wrf.ALL_TIMES, units='m s-1')[0,:]

#  Interpolate wind speeds to 100m height
wind_speed100 = wrf.interplevel(wspd, height, 100)

# Calculate wind power per square meter (units='W m-2')
air_density = 1000 
wpd = 0.5 * air_density * (wind_speed100) ** 3 

In [7]:
met_data['ghi'] = ghi
met_data['wind_speed100'] = wind_speed100
met_data['wpd'] = wpd
met_data

<xarray.Dataset>
Dimensions:         (Time: 40, bottom_top: 35, south_north: 191, west_east: 191)
Coordinates:
  * Time            (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    lon             (south_north, west_east) float32 -95.43335 ... -64.11832
    lat             (south_north, west_east) float32 30.952759 ... 47.507103
    wspd_wdir       <U4 'wspd'
    XLONG           (south_north, west_east) float32 -95.43335 ... -64.11832
    XLAT            (south_north, west_east) float32 30.952759 ... 47.507103
    XTIME           (Time) float32 0.0 60.0 120.0 180.0 ... 2220.0 2280.0 2340.0
    level           int64 100
Dimensions without coordinates: bottom_top, south_north, west_east
Data variables:
    Times           (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    temp_air        (Time, south_north, west_east) float32 308.1148 ... 292.3...
    cloud_fraction  (Time, bottom_top, south_north, west_east) float32 0.0 .....
    cos_zenith      (Time, south_north, west_east) float32 0.0 ... 0.84530807
    dni             (Time, south_north, west_east) float32 0.0 0.0 ... 943.1471
    dhi             (Time, south_north, west_east) float32 0.0 0.0 ... 53.327393
    height_agl      (Time, bottom_top, south_north, west_east) float32 10.885...
    ghi             (Time, south_north, west_east) float32 0.0 0.0 ... 850.5772
    wind_speed100   (Time, south_north, west_east) float32 2.566039 ... 6.393...
    wpd             (Time, south_north, west_east) float32 8448.114 ... 13069...
Attributes:
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      2011-07-17_00:00:00
    SIMULATION_START_DATE:           2011-07-17_00:00:00
    WEST-EAST_GRID_DIMENSION:        192
    SOUTH-NORTH_GRID_DIMENSION:      192
    BOTTOM-TOP_GRID_DIMENSION:       36
    DX:                              12000.0
    DY:                              12000.0
    AERCU_OPT:                       0
    AERCU_FCT:                       1.0
    IDEAL_CASE:                      0
    DIFF_6TH_SLOPEOPT:               0
    AUTO_LEVELS_OPT:                 2
    DIFF_6TH_THRESH:                 0.1
    DZBOT:                           50.0
    DZSTRETCH_S:                     1.3
    DZSTRETCH_U:                     1.1
    SKEBS_ON:                        0
    SPEC_BDY_FINAL_MU:               1
    USE_Q_DIABATIC:                  0
    GRIDTYPE:                        C
    DIFF_OPT:                        1
    KM_OPT:                          4
    DAMP_OPT:                        0
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      19
    RA_LW_PHYSICS:                   4
    RA_SW_PHYSICS:                   4
    SF_SFCLAY_PHYSICS:               1
    SF_SURFACE_PHYSICS:              7
    BL_PBL_PHYSICS:                  8
    CU_PHYSICS:                      99
    SF_LAKE_PHYSICS:                 0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      0
    GRID_FDDA:                       0
    GFDDA_INTERVAL_M:                0
    GFDDA_END_H:                     0
    GRID_SFDDA:                      0
    SGFDDA_INTERVAL_M:               0
    SGFDDA_END_H:                    0
    HYPSOMETRIC_OPT:                 2
    USE_THETA_M:                     1
    GWD_OPT:                         0
    SF_URBAN_PHYSICS:                0
    SF_SURFACE_MOSAIC:               0
    SF_OCEAN_PHYSICS:                0
    SHCU_PHYSICS:                    0
    MFSHCONV:                        0
    FEEDBACK:                        1
    SMOOTH_OPTION:                   0
    SWRAD_SCAT:                      1.0
    W_DAMPING:                       0
    DT:                              30.0
    RADT:                            15.0
    BLDT:                            0.0
    CUDT:                            0.0
    AER_OPT:                         0
    SWINT_OPT:

In [8]:
# Fix a bug in how wrfout data is read in -- attributes must be strings to be written to NetCDF
for var in met_data.data_vars:
    try:
        met_data[var].attrs['projection'] = str(met_data[var].attrs['projection'])
    except KeyError:
        pass

In [9]:
# Fix another bug that creates a conflict in the 'coordinates' attribute
for var in met_data.data_vars:
    try:
        del met_data[var].attrs['coordinates'] 
    except KeyError:
        pass

In [10]:
met_data

<xarray.Dataset>
Dimensions:         (Time: 40, bottom_top: 35, south_north: 191, west_east: 191)
Coordinates:
  * Time            (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    lon             (south_north, west_east) float32 -95.43335 ... -64.11832
    lat             (south_north, west_east) float32 30.952759 ... 47.507103
    wspd_wdir       <U4 'wspd'
    XLONG           (south_north, west_east) float32 -95.43335 ... -64.11832
    XLAT            (south_north, west_east) float32 30.952759 ... 47.507103
    XTIME           (Time) float32 0.0 60.0 120.0 180.0 ... 2220.0 2280.0 2340.0
    level           int64 100
Dimensions without coordinates: bottom_top, south_north, west_east
Data variables:
    Times           (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    temp_air        (Time, south_north, west_east) float32 308.1148 ... 292.3...
    cloud_fraction  (Time, bottom_top, south_north, west_east) float32 0.0 .....
    cos_zenith      (Time, south_north, west_east) float32 0.0 ... 0.84530807
    dni             (Time, south_north, west_east) float32 0.0 0.0 ... 943.1471
    dhi             (Time, south_north, west_east) float32 0.0 0.0 ... 53.327393
    height_agl      (Time, bottom_top, south_north, west_east) float32 10.885...
    ghi             (Time, south_north, west_east) float32 0.0 0.0 ... 850.5772
    wind_speed100   (Time, south_north, west_east) float32 2.566039 ... 6.393...
    wpd             (Time, south_north, west_east) float32 8448.114 ... 13069...
Attributes:
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      2011-07-17_00:00:00
    SIMULATION_START_DATE:           2011-07-17_00:00:00
    WEST-EAST_GRID_DIMENSION:        192
    SOUTH-NORTH_GRID_DIMENSION:      192
    BOTTOM-TOP_GRID_DIMENSION:       36
    DX:                              12000.0
    DY:                              12000.0
    AERCU_OPT:                       0
    AERCU_FCT:                       1.0
    IDEAL_CASE:                      0
    DIFF_6TH_SLOPEOPT:               0
    AUTO_LEVELS_OPT:                 2
    DIFF_6TH_THRESH:                 0.1
    DZBOT:                           50.0
    DZSTRETCH_S:                     1.3
    DZSTRETCH_U:                     1.1
    SKEBS_ON:                        0
    SPEC_BDY_FINAL_MU:               1
    USE_Q_DIABATIC:                  0
    GRIDTYPE:                        C
    DIFF_OPT:                        1
    KM_OPT:                          4
    DAMP_OPT:                        0
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      19
    RA_LW_PHYSICS:                   4
    RA_SW_PHYSICS:                   4
    SF_SFCLAY_PHYSICS:               1
    SF_SURFACE_PHYSICS:              7
    BL_PBL_PHYSICS:                  8
    CU_PHYSICS:                      99
    SF_LAKE_PHYSICS:                 0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      0
    GRID_FDDA:                       0
    GFDDA_INTERVAL_M:                0
    GFDDA_END_H:                     0
    GRID_SFDDA:                      0
    SGFDDA_INTERVAL_M:               0
    SGFDDA_END_H:                    0
    HYPSOMETRIC_OPT:                 2
    USE_THETA_M:                     1
    GWD_OPT:                         0
    SF_URBAN_PHYSICS:                0
    SF_SURFACE_MOSAIC:               0
    SF_OCEAN_PHYSICS:                0
    SHCU_PHYSICS:                    0
    MFSHCONV:                        0
    FEEDBACK:                        1
    SMOOTH_OPTION:                   0
    SWRAD_SCAT:                      1.0
    W_DAMPING:                       0
    DT:                              30.0
    RADT:                            15.0
    BLDT:                            0.0
    CUDT:                            0.0
    AER_OPT:                         0
    SWINT_OPT:

In [11]:
met_data = xr.Dataset.reset_coords(met_data, ['wspd_wdir','XLONG', 'XLAT', 'XTIME', 'level'], drop=True)

In [12]:
met_data

<xarray.Dataset>
Dimensions:         (Time: 40, bottom_top: 35, south_north: 191, west_east: 191)
Coordinates:
  * Time            (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    lon             (south_north, west_east) float32 -95.43335 ... -64.11832
    lat             (south_north, west_east) float32 30.952759 ... 47.507103
Dimensions without coordinates: bottom_top, south_north, west_east
Data variables:
    Times           (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    temp_air        (Time, south_north, west_east) float32 308.1148 ... 292.3...
    cloud_fraction  (Time, bottom_top, south_north, west_east) float32 0.0 .....
    cos_zenith      (Time, south_north, west_east) float32 0.0 ... 0.84530807
    dni             (Time, south_north, west_east) float32 0.0 0.0 ... 943.1471
    dhi             (Time, south_north, west_east) float32 0.0 0.0 ... 53.327393
    height_agl      (Time, bottom_top, south_north, west_east) float32 10.885...
    ghi             (Time, south_north, west_east) float32 0.0 0.0 ... 850.5772
    wind_speed100   (Time, south_north, west_east) float32 2.566039 ... 6.393...
    wpd             (Time, south_north, west_east) float32 8448.114 ... 13069...
Attributes:
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      2011-07-17_00:00:00
    SIMULATION_START_DATE:           2011-07-17_00:00:00
    WEST-EAST_GRID_DIMENSION:        192
    SOUTH-NORTH_GRID_DIMENSION:      192
    BOTTOM-TOP_GRID_DIMENSION:       36
    DX:                              12000.0
    DY:                              12000.0
    AERCU_OPT:                       0
    AERCU_FCT:                       1.0
    IDEAL_CASE:                      0
    DIFF_6TH_SLOPEOPT:               0
    AUTO_LEVELS_OPT:                 2
    DIFF_6TH_THRESH:                 0.1
    DZBOT:                           50.0
    DZSTRETCH_S:                     1.3
    DZSTRETCH_U:                     1.1
    SKEBS_ON:                        0
    SPEC_BDY_FINAL_MU:               1
    USE_Q_DIABATIC:                  0
    GRIDTYPE:                        C
    DIFF_OPT:                        1
    KM_OPT:                          4
    DAMP_OPT:                        0
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      19
    RA_LW_PHYSICS:                   4
    RA_SW_PHYSICS:                   4
    SF_SFCLAY_PHYSICS:               1
    SF_SURFACE_PHYSICS:              7
    BL_PBL_PHYSICS:                  8
    CU_PHYSICS:                      99
    SF_LAKE_PHYSICS:                 0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      0
    GRID_FDDA:                       0
    GFDDA_INTERVAL_M:                0
    GFDDA_END_H:                     0
    GRID_SFDDA:                      0
    SGFDDA_INTERVAL_M:               0
    SGFDDA_END_H:                    0
    HYPSOMETRIC_OPT:                 2
    USE_THETA_M:                     1
    GWD_OPT:                         0
    SF_URBAN_PHYSICS:                0
    SF_SURFACE_MOSAIC:               0
    SF_OCEAN_PHYSICS:                0
    SHCU_PHYSICS:                    0
    MFSHCONV:                        0
    FEEDBACK:                        1
    SMOOTH_OPTION:                   0
    SWRAD_SCAT:                      1.0
    W_DAMPING:                       0
    DT:                              30.0
    RADT:                            15.0
    BLDT:                            0.0
    CUDT:                            0.0
    AER_OPT:                         0
    SWINT_OPT:                       1
    AER_TYPE:                        1
    AER_AOD550_OPT:                  1
    AER_ANGEXP_OPT:                  1
    AER_SSA_OPT:                     1
    AER_ASY_OPT:                     1
    AER_AOD550_VAL:                  0.12
    AER_ANGEXP_VAL:               

In [13]:
postwrf.process_wrfout_data_manual('/Users/swardy9230/Box Sync/01_Research/01_Renewable_Analysis/Wind Resource Analysis/wrfout/19mp4lw4sw7lsm8pbl99cu/',
                           'wrfout_d01_2011-07-17', save_file=False)

<xarray.Dataset>
Dimensions:         (Time: 40, bottom_top: 35, south_north: 191, west_east: 191)
Coordinates:
  * Time            (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    lon             (south_north, west_east) float32 -95.43335 ... -64.11832
    lat             (south_north, west_east) float32 30.952759 ... 47.507103
    XTIME           (Time) float32 0.0 60.0 120.0 180.0 ... 2220.0 2280.0 2340.0
    level           int64 100
Dimensions without coordinates: bottom_top, south_north, west_east
Data variables:
    Times           (Time) datetime64[ns] 2011-07-17 ... 2011-07-18T15:00:00
    temp_air        (Time, south_north, west_east) float32 34.964813 ... 19.2...
    cloud_fraction  (Time, bottom_top, south_north, west_east) float32 0.0 .....
    cos_zenith      (Time, south_north, west_east) float32 0.0 ... 0.84530807
    dni             (Time, south_north, west_east) float32 0.0 0.0 ... 943.1471
    dhi             (Time, south_north, west_east) float32 0.0 0.0 ... 53.327393
    height_agl      (Time, bottom_top, south_north, west_east) float32 10.885...
    wspd            (Time, bottom_top, south_north, west_east) float32 2.2962...
    wdir            (Time, bottom_top, south_north, west_east) float32 169.32...
    ghi             (Time, south_north, west_east) float32 0.0 0.0 ... 850.5772
    wind_speed100   (Time, south_north, west_east) float32 2.566039 ... 6.393...
    wpd             (Time, south_north, west_east) float32 8448.114 ... 13069...
Attributes:
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      2011-07-17_00:00:00
    SIMULATION_START_DATE:           2011-07-17_00:00:00
    WEST-EAST_GRID_DIMENSION:        192
    SOUTH-NORTH_GRID_DIMENSION:      192
    BOTTOM-TOP_GRID_DIMENSION:       36
    DX:                              12000.0
    DY:                              12000.0
    AERCU_OPT:                       0
    AERCU_FCT:                       1.0
    IDEAL_CASE:                      0
    DIFF_6TH_SLOPEOPT:               0
    AUTO_LEVELS_OPT:                 2
    DIFF_6TH_THRESH:                 0.1
    DZBOT:                           50.0
    DZSTRETCH_S:                     1.3
    DZSTRETCH_U:                     1.1
    SKEBS_ON:                        0
    SPEC_BDY_FINAL_MU:               1
    USE_Q_DIABATIC:                  0
    GRIDTYPE:                        C
    DIFF_OPT:                        1
    KM_OPT:                          4
    DAMP_OPT:                        0
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      19
    RA_LW_PHYSICS:                   4
    RA_SW_PHYSICS:                   4
    SF_SFCLAY_PHYSICS:               1
    SF_SURFACE_PHYSICS:              7
    BL_PBL_PHYSICS:                  8
    CU_PHYSICS:                      99
    SF_LAKE_PHYSICS:                 0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      0
    GRID_FDDA:                       0
    GFDDA_INTERVAL_M:                0
    GFDDA_END_H:                     0
    GRID_SFDDA:                      0
    SGFDDA_INTERVAL_M:               0
    SGFDDA_END_H:                    0
    HYPSOMETRIC_OPT:                 2
    USE_THETA_M:                     1
    GWD_OPT:                         0
    SF_URBAN_PHYSICS:                0
    SF_SURFACE_MOSAIC:               0
    SF_OCEAN_PHYSICS:                0
    SHCU_PHYSICS:                    0
    MFSHCONV:                        0
    FEEDBACK:                        1
    SMOOTH_OPTION:                   0
    SWRAD_SCAT:                      1.0
    W_DAMPING:                       0
    DT:                              30.0
    RADT:                            15.0
    BLDT:                            0.0
    CUDT:                            0.0
    AER_OPT:                         0
    SWINT_OPT:                       

In [ ]:
# # Write the processed data back to a wrfout NetCDF file
# new_filename_1 = './wrfout_processed_d01_2011-01-24_01:00:00.nc'
# met_data.to_netcdf(path=new_filename_1)